In [5]:
using SparseArrays
using JLD2
using FileIO
function mod_idx(idx,n)
    while idx < 1
        idx += n
    end
    while idx > n
        idx -= n
    end
    return idx
end

function diff_mat2d(nx,ny,along,odiff,oacc=4,load_from_file =false)
    """
    nx,ny : the number of points in each dimension
    along : derivative along x-axis(along=1) or y-axis(along=2)
    odiff : order of the derivative
    oacc  : order of accuracy
    load_from_file : try to load precaluclated matrices
    return: the difference matrix
    *note that the marix if for periodic boundary. 
    *TBD : we introduce ghost layers for other boundary conditions to save the labour of 
        modifying the difference matrices, and these layers should be reset after 
        each iteration in time
    """
    if load_from_file
        file_name = "diff_mat2d_nx$(nx)_ny$(ny)_along$(along)_odiff$(odiff)_oacc$(oacc).jld2"
        # @__FILE__ is the location for the current running file
        # dirname(LOC) get the parent directory of LOC
        directory = joinpath(dirname(dirname(@__FILE__)),"save_diff_mat")
        full_file_name = joinpath(directory,file_name)
        if isfile(full_file_name)
            diff_mat = JLD2.load(full_file_name,"diff_mat2d")
            return diff_mat
        end
    end
        
    N = nx*ny
    if oacc == 2
        cdiff_coeff = [0.0  -0.5  0.0   0.5   0.0;
                       0.0   1.0 -2.0   1.0   0.0]
    elseif oacc == 4
        cdiff_coeff = [1.0/12.0  -2.0/3.0  0.0      2.0/3.0  -1.0/12.0;
                      -1.0/12.0   4.0/3.0 -5.0/2.0 	4.0/3.0  -1.0/12.0]
    elseif oacc == 6
        cdiff_coeff = [-1.0/60.0  3.0/20.0  -3.0/4.0  0.0        3.0/4.0  -3.0/20.0  1.0/60.0;
                        1.0/90.0 -3.0/20.0 	 3.0/2.0 -49.0/18.0  3.0/2.0  -3.0/20.0  1.0/90.0]
    elseif oacc == 8
        cdiff_coeff = [1.0/280.0  -4.0/105.0  1.0/5.0  -4.0/5.0   0.0         4.0/5.0  -1.0/5.0  4.0/105.0  -1.0/280.0;
                      -1.0/560.0   8.0/315.0 -1.0/5.0   8.0/5.0  -205.0/72.0  8.0/5.0  -1.0/5.0  8.0/315.0  -1.0/560.0]
    else
        error("oacc = 2,4,6 or 8, no other values allowed")
    end
    nrow,ncol = size(cdiff_coeff)
    nbrs = ncol ÷ 2         # integer division
    diff_mat = spzeros(Float64, N, N)
    stencil_ind =  Array{Int64,1}(undef,ncol)

    #=
    # a special case for order of accuracy = 2
    for i in 1:nx
        for j in 1:ny   
            if along == 1 
                stencil_ind[3] = i + (j-1)*nx                  # center
                stencil_ind[2] = mod_idx(i - 1, nx) + (j-1)*nx # left one
                stencil_ind[1] = mod_idx(i - 2, nx) + (j-1)*nx # left two
                stencil_ind[4] = mod_idx(i + 1, nx) + (j-1)*nx # right one
                stencil_ind[5] = mod_idx(i + 2, nx) + (j-1)*nx # right two
            elseif along == 2
                stencil_ind[3] = i + (j-1)*nx                        # center
                stencil_ind[2] = i + (mod_idx(j - 1, ny) - 1)*nx     # upper one
                stencil_ind[1] = i + (mod_idx(j - 2, ny) - 1)*nx     # upper two
                stencil_ind[4] = i + (mod_idx(j + 1, ny) - 1)*nx     # lower one
                stencil_ind[5] = i + (mod_idx(j + 2, ny) - 1)*nx     # lower two
            else
                error("along = 1 or 2, no other values allowed")
            end
            for is in 1:5
                diff_mat[stencil_ind[3],stencil_ind[is]] = cdiff_coeff[odiff,is]
            end
        end
    end
    =#
    for i in 1:nx
        for j in 1:ny   
            if along == 1 
                stencil_ind[nbrs + 1] = i + (j-1)*nx                             # center
                for nl in 1:nbrs
                    stencil_ind[nbrs + 1 - nl] = mod_idx(i - nl, nx) + (j-1)*nx   # the point left to the center by nl unit
                end
                for nr in 1:nbrs
                    stencil_ind[nbrs + 1 + nr] = mod_idx(i + nr, nx) + (j-1)*nx  # right one
                end
            elseif along == 2
                stencil_ind[nbrs + 1] = i + (j-1)*nx                              # center
                for nu in 1:nbrs
                    stencil_ind[nbrs + 1 - nu] = i + (mod_idx(j - nu, ny) - 1)*nx  # the point up to the center by nu unit
                end
                for nd in 1:nbrs
                    stencil_ind[nbrs + 1 + nd] = i + (mod_idx(j + nd, ny) - 1)*nx # the point down to the center by nd unit
                end
            else
                error("along = 1 or 2, no other values allowed")
            end 
            for is in 1:ncol
                diff_mat[stencil_ind[nbrs + 1],stencil_ind[is]] = cdiff_coeff[odiff,is]
            end
        end
    end
    dropzeros!(diff_mat)
    if load_from_file
        # save the calculated matix for future use
        file_name = "diff_mat2d_nx_$(nx)_ny$(ny)_along$(along)_odiff$(odiff)_oacc$(oacc).jld2"
        # @__FILE__ is the location for the current running file
        # dirname(LOC) get the parent directory of LOC
        directory = joinpath(dirname(dirname(@__FILE__)),"save_diff_mat")
        full_file_name = joinpath(directory,file_name)
        if isdir(directory)
            FileIO.save(full_file_name,"diff_mat2d",diff_mat)
        else
            mkdir(directory)
            JLD2.save(full_file_name,"diff_mat2d",diff_mat)
        end
    end
    return sparse(diff_mat)
end

diff_mat2d (generic function with 3 methods)

In [11]:
# Test the difference matrices
# include("./src/finite_diff.jl")
using Printf
# pbc f(0,y) = f(x_max,y),f(x,0) = f(x,y_max)
# descritization: delta_x = x_max/nx, delta_y = y_max/ny
# descretised field f[1 2... nx, 1 2... ny] 
# pbc: f[0,...] = f[nx,...] and f[...,0]=f[...,ny]
# f[0,...] and f[... ,0] are not included in the lattice 

x_max = y_max = 2.0*pi
nx = 64
ny = 64
N = nx*ny
delta_x = x_max/nx
delta_y = y_max/ny

x = range(delta_x, stop=2*pi, length=nx)
y = transpose(range(delta_y, stop=2*pi, length=ny))
f = @. sin(x)*cos(2.0*y)

# 0th order derivative
exact_derivatives = Dict((0,0)=>f)
# first derivatives(exact)
exact_derivatives[(1,0)] = @. cos(x)*cos(2.0*y)      #df/dx
exact_derivatives[(0,1)] = @. -2.0*sin(x)*sin(2.0*y) # dy/dx
# second derivatives(exact)
exact_derivatives[(2,0)] = @. -sin(x)*cos(2.0*y)     #d2f/dx2
exact_derivatives[(0,2)] = @. -4.0*sin(x)*cos(2.0*y) #d2d/dy2
exact_derivatives[(1,1)] = @. -2.0*cos(x)*sin(2.0*y) #d2/dxdy
# third derivatives(exact)
exact_derivatives[(3,0)] = @. -cos(x)*cos(2.0*y)     #d3f/dx3
exact_derivatives[(2,1)] = @. 2.0*sin(x)*sin(2.0*y) #d/dy d2f/dx2 
exact_derivatives[(1,2)] = @. -4.0*cos(x)*cos(2.0*y) #d/dx d2f/dy2 
exact_derivatives[(0,3)] = @. 8.0*sin(x)*sin(2.0*y)  #d3f/dy3 
# fourth derivatives(exact)
exact_derivatives[(4,0)] = @. sin(x)*cos(2.0*y)       #d4f/dx4
exact_derivatives[(3,1)] = @. 2.0*cos(x)*sin(2.0*y)   #d/dy d3f/dx3 
exact_derivatives[(2,2)] = @. 4.0*sin(x)*cos(2.0*y)   #d^2/dx^2 d2f/dy2 
exact_derivatives[(1,3)] = @. 8.0*cos(x)*sin(2.0*y)  #d^4f/dy4
exact_derivatives[(0,4)] = @. 16.0*sin(x)*cos(2.0*y)  #d^4f/dy4
oacc = 4
println(size(exact_derivatives[(0,4)]))
for odx in 0:2
    for ody in 0:2
        if odx + ody <= 4 && odx + ody > 0
            if odx == 0
                Dx = 1.0
            else
                Dx = diff_mat2d(nx,ny,1,odx,oacc)/(delta_x^odx)
            end
            if ody == 0
                Dy = 1.0
            else
                Dy = diff_mat2d(nx,ny,2,ody,oacc)/(delta_y^ody)
            end
            cdiff = Dx*Dy*reshape(f,(N,1))
            cdiff = reshape(cdiff,(nx,ny))
            rmse = sqrt(1.0/N*sum((cdiff - exact_derivatives[(odx,ody)]).^2))
            @printf("RMSE for [d/dx]^%i [d/dy]^%i is %1.8f \n",odx,ody,rmse)
        end
    end
end


(64, 64)
RMSE for [d/dx]^0 [d/dy]^1 is 0.00004932 
RMSE for [d/dx]^0 [d/dy]^2 is 0.00003292 
RMSE for [d/dx]^1 [d/dy]^0 is 0.00000155 
RMSE for [d/dx]^1 [d/dy]^1 is 0.00005241 
RMSE for [d/dx]^1 [d/dy]^2 is 0.00003910 
RMSE for [d/dx]^2 [d/dy]^0 is 0.00000052 
RMSE for [d/dx]^2 [d/dy]^1 is 0.00005035 
RMSE for [d/dx]^2 [d/dy]^2 is 0.00003498 


In [23]:
# Test the numerical integration using Simpson's rule
include("./src/finite_diff.jl")
using Printf
x = collect(range(0.0,1.0;length=20))
y = 4.0*x.^3
s = simpson_int1d(y,(0.0,1.0))
@printf("int 4x^3 from 0 to 1 is %1.8f, error is %1.8f\n",s,abs(1.0-s))

x  = collect(range(0.0,1.0*pi;length=20))
y = sin.(x)
s = simpson_int1d(y,(0.0,1.0*pi))
@printf("int sin(x) from 0 to pi is %1.8f, error is %1.8f\n",s,abs(2.0-s))

N = 100
x = collect(range(-4.0,4.0;length=N))
y = collect(range(-4.0,4.0;length=N))
z = exp.(-x.*x .- y'.*y').*(x.^2 .+ y.^2')'.*(x.^2 .+ y.^2')
@printf(" %1.8f\n",sum(z)*64/(N*N))

s = simpson_int2d(z,(-4.0,4.0),(-4.0,4.0))
@printf("int exp(-(x^2+y^2))*(x^2+y^2) in [-4,4]*[-6,6] %1.8f, error is %1.8f\n",s,abs(pi-s))




int 4x^3 from 0 to 1 is 0.93536217, error is 0.06463783
int sin(x) from 0 to pi is 1.99544132, error is 0.00455868
 3.07907265
int exp(-(x^2+y^2))*(x^2+y^2) in [-4,4]*[-6,6] 3.14158892, error is 0.00000374


In [120]:
# test the correlation integral
include("./src/finite_diff.jl")
Nw = 30
rx = collect(range(-5.0,5.0;length=2*Nw+1))
ry = collect(range(-5.0,5.0;length=2*Nw+1))
k = exp.(-rx.^2 .- (ry.^2)')

dx = 10.0/(2*Nw)
dy = 10.0/(2*Nw)

xx = collect(dx:dx:20.0)
yy = collect(dy:dy:20.0)
g = (10 .- xx).^2 .+ ((10 .- yy).^2)'

# the index of point (10,10)
cx,cy = trunc(Int,10/dx),trunc(Int,10/dy)+1
println(k[Nw+1,Nw+1])
s = corr2d(k,g,(cx,cy),dx,dy)
i = exp.(-g).*g 

#@printf("%1.8f\n",sum(i)*dx*dy)

#@printf("%1.8f\n",simpson_int2d(i,(0.0,20),(0.0,20)))
# the correlation integral is int dxdy exp((10-x)^2+(10-y)^2))*((10-x)^2+(10-y)^2)
@printf("int dxdy exp((10-x)^2+(10-y)^2))*((10-x)^2+(10-y)^2) %1.8f, error is %1.8f\n",s,abs(pi-s))



1.0
int exp(-(0-y^2))y^2  3.14159265, error is 0.00000000


In [17]:
# test the mean field model
include("./src/mean_field_model_gradient.jl")
using Printf
x_max = y_max = 2.0*pi
nx = ny = 256
N = nx*ny
T = 0.1
A = 0.1
B = 0.01
C = 0.01
K = 0.01
Gamma = 1.0
dt = 0.01

model = NumericalMeanField2D(x_max, y_max, nx, ny, dt)


set_model_params(model, T, A, B, C, K)
x = model.x
y = model.y'
rho = @. sin(x) * cos(2.0 * y)
rho_dx = @. cos(x) * cos(2.0 * y)
rho_dy = @. -2.0 * sin(x) * sin(2.0 * y)

rho_dxy = @. -2.0 * cos(x) * sin(2.0 * y)
rho_dxx = @. -sin(x) * cos(2.0 * y)
rho_dyy = @. -4.0 * sin(x) * cos(2.0 * y)
mu = @. 2.0*A * rho + 3.0*B * rho^ 2 - 2.0*K * (-5.0 * sin(x) * cos(2.0 * y))

j = zeros(Float64, 2, nx, ny)
j[1, :, :] = @. rho * (2.0*A * rho_dx + 6.0 * B * rho * rho_dx - 2.0*K * (-5.0 * cos(x) * cos(2.0 * y))) + T*rho_dx
j[2, :, :] = @. rho * (2.0*A * rho_dy + 6.0 * B * rho * rho_dy - 2.0*K * (10.0 * sin(x) * sin(2.0 * y))) + T*rho_dy
f = zeros(Float64, 2, nx, ny)

#sigma_xx = @. -C*(sin(x) * cos(2.0 * y))^2
#sigma_xy = @. -C*0.5 * sin(2.0 * x) * sin(4.0 * yprintln(rho))
#sigma_yy = @. -C*4.0 * (sin(x) * cos(2.0 * y))^2

f = zeros(Float64, 2, nx, ny)
f[1, :, :] = @. 2.0*C*(-sin(2.0 * x) * (cos(2.0 * y)^2) - 2.0 * sin(2.0 * x) * cos(4.0 * y))
f[2, :, :] = @. 2.0*C*(-cos(2.0 * x)* sin(4.0 * y) + 8.0 * sin(x)^2 * sin(4.0 * y))

set_initial_condition(model,rho)
update_parallel!(model)
rho_fd = reshape(model.rho,nx,ny)
rmse = sqrt(1.0/N*sum((rho_fd - rho).^2))
@printf("RMSE for rho is %1.8f \n",rmse)

# rmse of mu
mu_fd = reshape(model.mu,nx,ny)
rmse_mu = sum(abs.(mu_fd - mu))/sum(abs.(mu))
@printf("RMSE for mu is %1.8f \n",rmse_mu)
println(mu_fd[10,10])
println(mu[10,10])

# rmse of j
for alpha in 1:2
    j_fd = reshape(model.j[alpha, :, :],(nx,ny))
    rmse_j = sum(abs.(j_fd - j[alpha, :, :]))/sum(abs.(j[alpha, :, :]))
    @printf("RMSE for the %i th component of flux, j, is %1.8f \n",alpha,rmse_j)
end

# rmse of f
for alpha in 1:2
    f_fd = reshape(model.f[alpha, :, :],(nx,ny))
    rmse_f = sum(abs.(f_fd - f[alpha, :, :]))/sum(abs.(f[alpha, :, :]))
    @printf("RMSE for the %i th component of force density, f, is %1.8f \n",alpha,rmse_f)
end


LoadError: LoadError: invalid redefinition of constant NumericalMeanField2D
in expression starting at /Users/satyamanand/Desktop/Bromf-main/src/mean_field_model_gradient.jl:12